# k8s工作负载

## 工作负载cronjob

参考文献：https://kubernetes.io/zh/docs/tasks/job/automated-tasks-with-cron-jobs/

生成cronjob.yaml模板文件

```shell
# 生成yaml文件
k create cronjob hello --image=busybox --schedule="0/10 * * * *" --dry-run='client' -o yaml
```

`hello-cronjob.yaml`

```yaml
apiVersion: batch/v1beta1
kind: CronJob
metadata:
  creationTimestamp: null
  name: hello
spec:
  jobTemplate:
    metadata:
      creationTimestamp: null
      name: hello
    spec:
      template:
        metadata:
          creationTimestamp: null
        spec:
          containers:
          - image: busybox
            name: hello
            resources: {}
            command:
            - /bin/sh
            - -c
            - date; echo Hello from the Kubernetes cluster
          restartPolicy: OnFailure
  schedule: 0/10 * * * *
status: {}
```

> 增加`spec.jobTemplate.spec.containers.command` `/bin/sh -c date; echo Hello from Kubernetes cluster`

创建cronjob:

```shell
kubectl apply -f hello-cronjob.yaml
```

查看cronjob

```shell
kubectl get cronjob hello
```

输出类似于

```
NAME    SCHEDULE       SUSPEND   ACTIVE   LAST SCHEDULE   AGE
hello   0/10 * * * *   False     0        <none>          88s
```

删除cronjob

```shell
kubectl delete cronjob hello
```

cron时间表语法

```
# ┌───────────── 分钟 (0 - 59)
# │ ┌───────────── 小时 (0 - 23)
# │ │ ┌───────────── 月的某天 (1 - 31)
# │ │ │ ┌───────────── 月份 (1 - 12)
# │ │ │ │ ┌───────────── 周的某天 (0 - 6)（周日到周一；在某些系统上，7 也是星期日）
# │ │ │ │ │
# │ │ │ │ │
# │ │ │ │ │
# * * * * *
```

参考文献：https://crontab.guru/#

cron调度示例

|输入|描述|相当于|
|----|---|-----|
|@yearly (or @annually)|	每年 1 月 1 日的午夜运行一次	|0 0 1 1 *|
|@monthly	|每月第一天的午夜运行一次|	0 0 1 * *|
|@weekly|	每周的周日午夜运行一次	|0 0 * * 0|
|@daily (or @midnight)	|每天午夜运行一次	|0 0 * * *|
|@hourly|	每小时的开始一次|	0 * * * *|
|@minute|每分钟执行一次|0/10 * * * * |

## FAQ

### Q：cron每天定时执行任务，任务执行时间不准确？

A：`kube-controller-manager`pod的时区与主机Node上的时区不一致导致。

修改`kube-controller-manager.yaml`配置文件`sudo vim /etc/kubernetes/manifests/kube-controller-manager.yaml`，将Node上的`/etc/localtime`挂载controller-manager的pod中。

```yaml
    volumeMounts:
    - mountPath: /etc/ssl/certs
      name: ca-certs
      readOnly: true
    - name: localtime
      mountPath: /etc/localtime
      readOnly: true
  volumes:
  - hostPath:
      path: /etc/ssl/certs
      type: DirectoryOrCreate
    name: ca-certs
  - hostPath:
      path: /etc/localtime
    name: localtime
```